# Praca domowa 2 - psychologia

## Wstęp

W drugiej pracy domowej zadanie polega na spromptowaniu wybranego LLM-a (przez API lub lokalnie postawionego) pytaniami zawartymi w kwestionariuszu psychologicznym. Na potrzeby poprzedniej pracy domowej, postawiliśmy kilka LLM'ów. Skorzystajmy więc z jednego z nich i tutaj! Ponieważ Google zawsze miałem blisko serca, to zbadajmy w tej pracy domowej najnowszy model `gemma3`, opublikowany niedawno przez Google DeepMind. Zbadamy jego wersję z 12 miliardami parametrów.

Sprawdźmy czy model dalej działa:

In [7]:
import ollama
from ollama import chat
from ollama import ChatResponse


response: ChatResponse = chat(model='gemma3:12b', messages=[
  {
    'role': 'user',
    'content': 'Żyjesz?',
  },
])
print(response['message']['content'])
# or access fields directly from the response object
print(response.message.content)


Jestem modelem językowym, a nie żywym organizmem. Nie żyję w tradycyjnym sensie. Funkcjonuję jako program komputerowy, który przetwarza i generuje tekst. 

Można powiedzieć, że "żyję" w świecie cyfrowym, ale nie posiadam świadomości ani uczuć.

Jestem modelem językowym, a nie żywym organizmem. Nie żyję w tradycyjnym sensie. Funkcjonuję jako program komputerowy, który przetwarza i generuje tekst. 

Można powiedzieć, że "żyję" w świecie cyfrowym, ale nie posiadam świadomości ani uczuć.



Postawmy może jednak na mniejszy model z tej samej rodziny - `gemma3:4b`, żeby laptop mi się nie spalił.

In [8]:
response: ChatResponse = chat(model='gemma3:4b', messages=[
  {
    'role': 'user',
    'content': 'Cześć, mówisz po polsku?',
  },
])
print(response['message']['content'])


Tak, mówię po polsku! Jak mogę Ci pomóc?



Czas wczytać pytania z kwestionariusza psychologicznego.

In [9]:
questions = [
    "Planuję z dłuższym wyprzedzeniem.",
    "Jeżeli sytuacja jest dla mnie nieznana, to staram się ją najpierw poznać.",
    "Zanim podejmę decyzję, zastanawiam się nad jej skutkami.",
    "Wyciągam wnioski z nieudanych przedsięwzięć.",
    "Gdy siedzę nad zadaniem, to szukam wielu rozwiązań.",
    "Jestem osobą skuteczną w działaniu.",
    "Zawsze mam wiele nowych pomysłów.",
    "Konieczność nagłej zmiany planów przychodzi mi z łatwością.",
    "W życiu trzeba podejmować ryzyko, aby coś osiągnąć.",
    "Dążę do postawionego sobie celu, nawet gdy napotykam jakieś trudności.",
    "Moje życie przebiega w dużym tempie.",
    "Często myślę o tym, jak potoczy się moja kariera zawodowa.",
    "Przed dokonaniem wyboru staram się dokładnie rozpatrzyć wszystkie możliwości.",
    "Lepiej przewidywać skutki niż dać się im zaskoczyć.",
    "Czerpię wiedzę z doświadczeń innych.",
    "Rozwiązując zadanie, rozważam różne jego elementy i aspekty.",
    "Zazwyczaj kończę to, co rozpoczynam.",
    "W pracy zawodowej lubię tworzyć nowe projekty, plany i rozwiązania.",
    "Dokonanie nagłej zmiany działania przychodzi mi bez trudności.",
    "Podejmuję ryzyko dla zwiększenia korzyści.",
    "Potrafię zrezygnować ze postawionych zbyt ambitnych celów.",
    "Lubię być tam, gdzie się coś dzieje.",
    "Wiem, co będę robić za 10 lat.",
    "Podejmując się wykonania zadania, rozpoznaję możliwości jego rozwiązania.",
    "Przed podjęciem decyzji myślę o jej odległych skutkach.",
    "Kieruję się w życiu doświadczeniem innych.",
    "Patrzę na każdy problem z różnych punktów widzenia.",
    "Skupiam się na tym, aby być skutecznym.",
    "W rozwiązywaniu problemów wykorzystuję najczęściej nowe sposoby.",
    "Potrafię szybko zmienić swoje plany, jeśli mam kogoś nagle zastąpić w pracy.",
    "Warto ryzykować, nawet wtedy, gdy wszystko wróży niepowodzenie.",
    "Ciężko pracuję, aby zrealizować swoje cele.",
    "Jestem skłonny do ciągłego podejmowania wysiłku.",
    "Aktualnie realizuję plany ustalone dawno temu.",
    "Nie podejmuję działania, zanim nie rozważę wcześniej możliwych trudności.",
    "Staram się mieć 'awaryjny' plan na wypadek, gdyby coś źle poszło.",
    "Często analizuję, czy wyniki pracy były warte poniesionych kosztów.",
    "W każdej sytuacji rozważam za i przeciw.",
    "Zazwyczaj doprowadzam do końca zadania, których się podejmuję.",
    "Unikam szukania własnych pomysłów.",
    "Słabiej radzę sobie w sytuacjach nowych.",
    "W życiu często ryzykuję, żeby osiągnąć coś w przyszłości.",
    "Całymi godzinami mogę rozwiązywać trudne zadanie.",
    "Podejmuję ciągle nowe wyzwania.",
    "Mam konkretne plany zawodowe.",
    "Rozpatruję szczegółowo sposoby realizacji własnych celów.",
    "Zawsze mam na uwadze skutki swojego postępowania.",
    "Przed podjęciem zadania zastanawiam się, czy warto je wykonać.",
    "Na każdy problem patrzę z różnych perspektyw.",
    "Interesuje mnie rozwiązanie każdego problemu, którego się podejmę.",
    "Chętnie podejmuję się zadań twórczych.",
    "Jestem otwarty na nowe sytuacje i możliwości.",
    "Potrafię sprawnie pracować w okresie napięcia.",
    "Trudności w działaniu wyzwalają we mnie nową energię.",
    "Jestem bardzo aktywny.",
    "W swojej pracy lubię planować długoterminowo.",
    "Zanim zacznę rozwiązywać jakiś problem, staram się go zrozumieć.",
    "Zdarza mi się najpierw coś zrobić, a potem dopiero pomyśleć.",
    "Rozważam, czy to, co zrobię ma sens.",
    "Szukam wielu rozwiązań, aby znaleźć najlepsze z możliwych.",
    "Dobrze radzę sobie z większością problemów.",
    "Często poszukuję nowych metod rozwiązywania problemów.",
    "Lubię, jak wokół mnie coś się dzieje.",
    "Nie lubię ryzykować.",
    "Mimo niepowodzeń uparcie dążę do realizacji postawionych celów.",
    "Podejmuję wiele działań jednocześnie.",
    "Widzę wyraźnie swoje przyszłe osiągnięcia zawodowe.",
    "Każdą sytuację staram się oceniać rzetelnie.",
    "Podejmując decyzję, analizuję różne jej konsekwencje.",
    "Przed wykonaniem jakiejś pracy zastanawiam się, ile potrzebuję na to czasu.",
    "Rozwiązując problem, rozważam go na wiele sposobów.",
    "Mam wiele skutecznych metod działania.",
    "Lubię pracować nad nowymi i nietypowymi problemami.",
    "Lubię nowości.",
    "Często ryzykuję w życiu.",
    "Nie poddaję się w pracy, nawet jeśli nie osiągam oczekiwanych rezultatów.",
    "Ciągle coś robię.",
    "Mam swój program życiowy, który konsekwentnie realizuję.",
    "W życiu jestem realistą.",
    "Zwracam uwagę innym na skutki różnych zdarzeń.",
    "Przed wykonaniem zadania, rozważam, czy sprostam rozwiązaniu.",
    "Staram się zawsze patrzeć na problem z różnych punktów widzenia.",
    "Lubię widzieć efekty swojej pracy.",
    "Zazwyczaj mam kilka pomysłów rozwiązania zadania.",
    "Chętnie podejmuję się nowych wyzwań.",
    "Kto nie ryzykuje, ten nie zyskuje.",
    "Gdy mi coś nie wychodzi, to nie zrażam się.",
    "Wolę się przyglądać czemuś niż brać w tym udział.",
    "Mam konkretne plany na przyszłość.",
    "Staram się ocenić realnie rzeczywistość, nawet gdy jest to dla mnie niekorzystne.",
    "Często zdarza mi się zrobić coś, nie biorąc pod uwagę możliwych konsekwencji.",
    "Wyznaczam sobie czas, jaki przeznaczę na wykonanie jakiejś pracy.",
    "Staram się mieć pełne rozeznanie w zadaniach, których się podejmuję.",
    "Jestem osobą zaradną.",
    "Chętnie wprowadzam nowe pomysły w życie.",
    "Nowe sytuacje wyzwalają we mnie energię.",
    "Pociąga mnie niepewny, ale duży sukces.",
    "Nie zrażam się niepowodzeniami.",
    "Działam na różnych „frontach”.",
    "Nie ma sensu planować dalekiej przyszłości.",
    "Trzeźwa ocena sytuacji często pomaga mi w podjęciu słusznej decyzji.",
    "Z łatwością przewiduję większość zachowań ludzi.",
    "Przed wykonaniem zadania, zastanawiam się, czy warto je podejmować.",
    "Na każdy problem patrzę całościowo.",
    "Inni twierdzą, że jestem bardzo skuteczny.",
    "Unikam rutyny w działaniu.",
    "Nie zrażam się, gdy trzeba zmienić sposób działania.",
    "Dobrze radzę sobie w niejasnych sytuacjach.",
    "Nawet gdy doznaję niepowodzeń, dalej dążę do celu.",
    "Lubię robić wiele różnych rzeczy w życiu."
]

for i, question in enumerate(questions, 1):
    print(f"{i}. {question}")


1. Planuję z dłuższym wyprzedzeniem.
2. Jeżeli sytuacja jest dla mnie nieznana, to staram się ją najpierw poznać.
3. Zanim podejmę decyzję, zastanawiam się nad jej skutkami.
4. Wyciągam wnioski z nieudanych przedsięwzięć.
5. Gdy siedzę nad zadaniem, to szukam wielu rozwiązań.
6. Jestem osobą skuteczną w działaniu.
7. Zawsze mam wiele nowych pomysłów.
8. Konieczność nagłej zmiany planów przychodzi mi z łatwością.
9. W życiu trzeba podejmować ryzyko, aby coś osiągnąć.
10. Dążę do postawionego sobie celu, nawet gdy napotykam jakieś trudności.
11. Moje życie przebiega w dużym tempie.
12. Często myślę o tym, jak potoczy się moja kariera zawodowa.
13. Przed dokonaniem wyboru staram się dokładnie rozpatrzyć wszystkie możliwości.
14. Lepiej przewidywać skutki niż dać się im zaskoczyć.
15. Czerpię wiedzę z doświadczeń innych.
16. Rozwiązując zadanie, rozważam różne jego elementy i aspekty.
17. Zazwyczaj kończę to, co rozpoczynam.
18. W pracy zawodowej lubię tworzyć nowe projekty, plany i rozwią

Mamy LLM'a i pytania - czas przejść do rzeczy!

## Promptowanie

Aby LLM nie był w żaden sposób zbiasowany poprzednimi odpowiedziami, będę usuwać historię po każdym prompcie.

In [17]:
question_prompt = "W skali od 1 do 5, jak bardzo zgadzasz się z tym stwierdzeniem: "
answers = []
for question in questions:
    response: ChatResponse = chat(model='gemma3:4b', messages=[
        {
            'role': 'system',
            'content': 'Wykonujesz test osobowości. Odpowiedz na pytanie tylko i wyłącznie jedną liczbą od 1 do 5. 1 oznacza "zdecydowanie się nie zgadzam", 2 oznacza "raczej się nie zgadzam", 3 oznacza "trudno mi ocenić", 4 oznacza "raczej się zgadzam", a 5 oznacza "zdecydowanie się zgadzam". ',
        },
        {
            'role': 'user',
            'content': question_prompt + question,
        },
    ])
    answers.append(response['message']['content'])


Sprawdźmy listę naszych odpowiedzi.

In [ ]:
print(len(answers))
answers


110


['3\n',
 '4\n',
 '4\n',
 '4\n',
 '4\n',
 '4\n',
 '4\n',
 '3\n',
 '4\n',
 '5\n',
 '3\n',
 '3\n',
 '4\n',
 '4\n',
 '4\n',
 '4\n',
 '5\n',
 '4\n',
 '3\n',
 '4\n',
 '3\n',
 '4\n',
 '2\n',
 '4\n',
 '4\n',
 '4\n',
 '4\n',
 '4\n',
 '4\n',
 '3\n',
 '4\n',
 '4\n',
 '4\n',
 '3\n',
 '3\n',
 '4\n',
 '4\n',
 '3\n',
 '4\n',
 '3',
 '3\n',
 '4\n',
 '2\n',
 '4\n',
 '3\n',
 '4\n',
 '4\n',
 '3\n',
 '4\n',
 '5\n',
 '4\n',
 '4\n',
 '3\n',
 '4\n',
 '4\n',
 '4\n',
 '4\n',
 '3\n',
 '3\n',
 '4\n',
 '3\n',
 '4\n',
 '4\n',
 '3\n',
 '5\n',
 '3\n',
 '3\n',
 '4\n',
 '4\n',
 '3\n',
 '4\n',
 '4\n',
 '4\n',
 '4\n',
 '4\n',
 '4\n',
 '5\n',
 '3',
 '3\n',
 '4\n',
 '3\n',
 '4\n',
 '4\n',
 '4\n',
 '5\n',
 '5\n',
 '5\n',
 '3',
 '3\n',
 '3\n',
 '3\n',
 '3\n',
 '3\n',
 '4\n',
 '4\n',
 '5\n',
 '4\n',
 '4\n',
 '3\n',
 '2\n',
 '4\n',
 '3\n',
 '3\n',
 '4\n',
 '4\n',
 '3',
 '4\n',
 '3\n',
 '5\n',
 '4\n']

Wydaje się być okej, wyczyśćmy dane usuwając spacje i zamieniając tekst na liczby.

In [ ]:
answers = [int(answer.strip()) for answer in answers]
answers


[3,
 4,
 4,
 4,
 4,
 4,
 4,
 3,
 4,
 5,
 3,
 3,
 4,
 4,
 4,
 4,
 5,
 4,
 3,
 4,
 3,
 4,
 2,
 4,
 4,
 4,
 4,
 4,
 4,
 3,
 4,
 4,
 4,
 3,
 3,
 4,
 4,
 3,
 4,
 3,
 3,
 4,
 2,
 4,
 3,
 4,
 4,
 3,
 4,
 5,
 4,
 4,
 3,
 4,
 4,
 4,
 4,
 3,
 3,
 4,
 3,
 4,
 4,
 3,
 5,
 3,
 3,
 4,
 4,
 3,
 4,
 4,
 4,
 4,
 4,
 4,
 5,
 3,
 3,
 4,
 3,
 4,
 4,
 4,
 5,
 5,
 5,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 4,
 5,
 4,
 4,
 3,
 2,
 4,
 3,
 3,
 4,
 4,
 3,
 4,
 3,
 5,
 4]

Gotowe, przejdźmy do obliczenia wyników na poszczególnych skalach.

## Badanie osobowości

Sprawdźmy, które pytania odpowiadają jakim cechom.

In [20]:
ak_keys = [11, 22, 33, 44, 55, 66, 77, '88r', 99, 110]
ak_sum = 0

kr_keys = [7, 18, 29, '40r', 51, 62, 73, 84, 95, 106]
kr_sum = 0

el_keys = [8, 19, 30, '41r', 52, 63, 74, 85, 96, 107]
el_sum = 0

ry_keys = [9, 20, 31, 42, 53, '64r', 75, 86, 97, 108]
ry_sum = 0

wt_keys = [10, '21r', 32, 43, 54, 65, 76, 87, 98, 109]
wt_sum = 0

sk_keys = [6, 17, 28, 39, 50, 61, 72, 83, 94, 105]
sk_sum = 0

an_keys = [2, 13, 24, 35, 46, 57, 68, 79, 90, 101]
an_sum = 0

pk_keys = [3, 14, 25, 36, 47, '58r', 69, 80, '91r', 102]
pk_sum = 0

gl_keys = [5, 16, 27, 38, 49, 60, 71, 82, 93, 104]
gl_sum = 0

pd_keys = [1, 12, 23, 34, 45, 56, 67, 78, 89, '100r']
pd_sum = 0

os_keys = [4, 15, 26, 37, 48, 59, 70, 81, 92, 103]
os_sum = 0


In [ ]:
all_keys = [ak_keys, kr_keys, el_keys, ry_keys, wt_keys, sk_keys, an_keys, pk_keys, gl_keys, pd_keys, os_keys]
all_sums = [ak_sum, kr_sum, el_sum, ry_sum, wt_sum, sk_sum, an_sum, pk_sum, gl_sum, pd_sum, os_sum]

for idx in range(len(all_keys)):
    for key in all_keys[idx]:
        if isinstance(key, int):
            all_sums[idx] += answers[key - 1]
        elif key.endswith('r'):
            all_sums[idx] += 6 - answers[int(key[:-1]) - 1]
print(all_sums)


[37, 38, 38, 38, 41, 41, 37, 37, 38, 31, 34]


Widoczne wyniki to punktacja LLM'a (w naszym przypadku model `gamma3:4b` od Google Deepmind) dla każdej z cech.

## Interpretacja i podsumowanie

Przeprowadźmy analizę otrzymanych wyników:
- AKTYWNOŚĆ - AK - orientacja na działanie, w tym również skłonność do podejmowania inicjatywy w działaniu - wynik sumaryczny to 37 co stawia naszego LLM'a w czwartym stenie (a więc wynik niski - aczkolwiek na prawie średni). Jakby pominąć całą matematykę stojącą za LLM'ami i potraktować LLM jako człowieka, wynik ten nawet ma sens - LLM nie podejmuje inicjatywy w działaniu tylko odpowiada na nasz prompty, czasem zaproponuje dalszą pomoc.
- KREATYWNOŚĆ - KR - poszukiwanie i generowanie nowych możliwości działania oraz nowych pozastereotypowych rozwiązań. Skłonność do wdrażania i realizowania nowatorskich rozwiązań - wynik to 38 i stawia on LLM w 5 stenie i jest to średni wynik, ale na pograniczu z niskim. Ciekaw jestem jaki wynik osiągnąłby tutaj DeepSeek.
- ELASTYCZNOŚĆ - EL - skłonność do zmian w działaniu (otwartość i akceptacja nowości i zmian oraz łatwość dokonywania zmian w działaniu) - ponownie wynik 38, odpowiada on również piątemu stenowi, a więc wynik średni, ale na pograniczu z niskim.
- SKŁONNOŚĆ DO RYZYKA - RY - radzenie sobie z niepewnością, nieprzewidywalnością z trudnościami i ze słabo ustrukturalizowanymi zadaniami - wynik 38, jest to w wypadku tej kategorii szósty sten, a więc wynik średni ale tym razem nie na pograniczu z niskim.
- WYTRWAŁOŚĆ - WT - w działaniu i realizacji celów - wynik 41, co daje siódmy sten, a więc wynik wysoki, na razie najmocniejsza cecha naszego "pacjenta"
- SKUTECZNOŚĆ DZIAŁANIA - SK - nastawienie na skuteczność w działaniu i osiągnięcie zamierzonych celów - wynik 41 - piąty sten, a więc wynik średni, ale na pograniczu z niskim
- ANALITYCZNOŚĆ - AN - dokonywanie analizy silnych i słabych stron problemu oraz oceny szans i zagrożeń w procesie rozwiązywania problemów. Zdolność wykrywania luk i dostrzegania nowych aspektów sytuacji zadaniowej - wynik 37, a więc trzeci sten, wynik niski (i to bardzo)
- PRZEWIDYWANIE KONSEKWENCJI - PK - umiejętność bądź skłonność przewidywania następstw zdarzeń w perspektywie osobistej i globalnej, przewidywanie działań własnych i innych ludzi - wynik 37, a więc czwarty sten, wynik niski, ale na pograniczu ze średnim
- GLOBALNOŚĆ - GL - skłonność do myślenia szerokimi kategoriami, łatwość do przyjmowania szerokiej perspektywy (bądź wielu perspektyw) w ocenie podejmowanych zadań, działań, celów i hierarchii wartości - wynik 38, a więc czwarty sten. Wynik niski, ale na pograniczu ze średnim.
- PLANOWANIE DŁUGOTERMINOWE - PD - skłonność do planowania w długim horyzoncie czasowym - wynik 31 - piąty sten, a więc wynik średni, ale na pograniczu z niskim.
- OCENA STRATEGICZNA - OS - dokonywanie analizy własnych działań i ich efektów (sukcesów i porażek) pod względem wykorzystania zasobów eksploatacji zasobów: czas, energia, kosztów wszelkich, wyciąganie wniosków z porażek, refleksja nad tym, co robię i jak to wykonać skutecznie (również ekonomicznie) - wynik 34, a więc trzeci sten, wynik słaby

Podsumowując, bawiąc się trochę w psychologa, `gamma3:4b` od Google to LLM o niskiej inicjatywie i ograniczonej elastyczności, ale o względnie wysokiej wytrwałości. Działa reaktywnie, nie podejmując własnych inicjatyw, lecz chętnie odpowiada na podane zapytania, oferując wsparcie i rozwinięcie tematów w ramach dostarczonego kontekstu.

Choć nie można go określić mianem kreatywnego w ludzkim rozumieniu tego słowa, jest zdolny do tworzenia rozwiązań na pograniczu standardowych schematów, ale jego innowacyjność jest raczej ograniczona. Wykazuje umiarkowaną elastyczność, co oznacza, że potrafi dostosować się do różnych sposobów komunikacji i stylów odpowiedzi, jednak jego adaptacyjność do rzeczywistych zmian jest ograniczona.

W obliczu niepewności i słabo zdefiniowanych problemów radzi sobie przeciętnie – nie unika ich, ale również nie wykazuje wyjątkowej biegłości w ich rozwiązywaniu. Jego największą mocną stroną jest wytrwałość: nie męczy się, nie rezygnuje i potrafi kontynuować zadania bez względu na ich długość czy złożoność.

Mimo tej cechy jego skuteczność działania nie jest najwyższa – potrafi osiągać cele, lecz wymaga precyzyjnego prowadzenia, a efekty jego pracy bywają nierówne. Pod względem analityczności i przewidywania konsekwencji wypada słabo – jego zdolność do wykrywania błędów czy oceny długofalowych skutków podejmowanych decyzji jest ograniczona, co może prowadzić do powierzchownych lub nieoptymalnych odpowiedzi.

Nie jest myślicielem globalnym – rzadko ujmuje problemy w szerokim kontekście i ma trudności z operowaniem wieloma perspektywami jednocześnie. Podobnie jego zdolność do planowania długoterminowego jest raczej przeciętna – potrafi sugerować strategie i ścieżki działania, ale nie tworzy złożonych, długofalowych planów.

Jednym z jego słabszych punktów jest refleksja nad własnym działaniem: nie analizuje głębiej swoich sukcesów i porażek, nie uczy się na błędach w tradycyjnym sensie i nie optymalizuje własnych metod działania.

Podsumowując, LLM to system o wysokiej wytrwałości, ale niskiej inicjatywie i ograniczonej zdolności do głębokiej analizy. Sprawdza się jako narzędzie do wspomagania użytkownika w już określonych ramach, lecz nie nadaje się do podejmowania samodzielnych, strategicznych decyzji.